# Tutorial
Cálculo de Indice de Kovats utilizando código programado en lenguaje Python

PARA COMENZAR: Instalar Jupyter Notebook
Pre requisito: Python
Para nuevos usuarios, se recomienda utilizar la distribución Anaconda, que convenientemente instala Python, Jupyter Notebook y otros paquetes de uso común

Pasos de instalación:
1.	Descargar Anaconda. Se recomienda instalar la última versión: https://www.anaconda.com/download/
2.	Instalar la versión de Anaconda descargada, siguiendo las instrucciones en la página de descarga.
3.	Ya has instalado Jupyter Notebook. Para abrirlo: abre la terminal “Anaconda Prompt”, escribe “jupyter notebook” y presiona Enter.


IMPORTANTE: Al abrir Jupyter Notebook, seleccionar la carpeta de trabajo! Es decir, aquella carpeta donde se encuentran los archivos Excel a utilizar.

- El símbolo # se ubica al inicio de una línea para que  no sea considerada como parte del código, siendo muy útil para agregar instrucciones o recordatorios
- Para correr un comando: hacer click en uno de los cuadros y presionar 'Shift+Enter'
- SÓLO se correrán las líneas que se encuentren en colores, es decir, aquellas que NO tengan el símbolo # al inicio

Cualquier cosa me preguntan!
Ale :)


In [3]:
#Paso 1: Importamos la librería pandas y numpy
import pandas as pd 
import numpy as np

In [5]:
#Paso 2: Cargar el archivo Excel donde se encuentra la tabla con cada peak y su respectivo tiempo de retención
#obtenida a partir de GCMS Postrun Analysis. Utilizar para este caso, el archivo excel Ejemplo.xlsx
datos = pd.read_excel('Ejemplo.xlsx')

#Si quiero visualizar las primeras 5 filas de la tabla para verificar que se encuentra correctamente cargada:
#datos.head()

#Alternativamente, podemos ver "N" filas utilizando:
#datos.head(N)

#Paso 3: Dado que en las tablas el tiempo se encuentra sin comas, dividir todos los tiempos de retención por 1000:
t = datos['Rt']/1000

#Exportar los tiempos de retención resultantes a un archivo Excel
t.to_excel('tiempos.xlsx')

#Paso 4: En la carpeta de trabajo, abrir el Excel tiempos.xlsx, copiar la columna de tiempos de retención y 
#pegar en la columna Rt del Excel original


In [10]:
#Paso 5: Seleccionar peaks de interés

#Si quiero seleccionar los peaks de tiempo de retención mayor o igual a 6.875 -¡RECOMENDABLE! Sólo desde ahí podemos calcular-
comp = datos[(datos['Rt']>=6.875)]

#Si quiero trabajar con todos los peaks:
#comp = datos

#Si quiero seleccionar los peaks de área mayor o igual a 0.5
#comp = datos[(datos['Area%']>=0.5)]

#Si quiero excluir los peaks que corresponden a sangrado de columna-> OJO, previamente hay que marcar en el Excel
#dichos peaks con la letra 'R' en una columna adicional, de nombre 'Class'
#comp = datos[(datos['Class']!='R')]

#Si quiero combinar estas condiciones:
#comp = datos[(datos['Area%']>=0.5) & (datos['Rt']>=6.875)]
#comp = datos[(datos['Area%']>=0.5) & (datos['Class']!='R') & (datos['Rt']>=6.875)]

#Si quiero observar los peaks seleccionados
#comp.head()

In [11]:
#Paso 6: Cargar archivo Excel donde se encuentran los datos para calcular Indice de Kovats
alc_mom = pd.read_excel('AlcMomiasPandita.xlsx')

#Para verificar que se encuentra cargado
#alc_mom.head()

In [12]:
#Paso 7: Correr ciclo 'for' que va tomando cada tiempo de retención, ubicándolo en un intervalo y calculando el IK

IK_teo = []
for Rt in comp['Rt']:
    for i in range(0, alc_mom.shape[0]):
        cota_inf = alc_mom['T b'][i]
        cota_sup = alc_mom['T a'][i]
       
        if Rt >= cota_inf and Rt < cota_sup:
            nb = alc_mom['nb'][i]
            log_rt = np.log10(Rt)
            log_tb = alc_mom['Log T b'][i]
            log_ta = alc_mom['Log Ta'][i]
        
            kovats = 100*(nb+((log_rt-log_tb)/(log_ta-log_tb)))
            IK_teo.append(kovats)
            

In [13]:
#Paso 8: Exportar tabla Excel con IK calculados

IK_teo_df = pd.DataFrame(IK_teo)
IK_teo_df.columns = ['IK teo']
IK_teo_df.head()

comp = comp.reset_index()
comp_new = pd.concat([comp, IK_teo_df], axis=1)
comp_new.to_excel('./Listeilor.xlsx')